In [ ]:
import os
import yaml

# For training
import torch
from ultralytics import YOLO



Recover variable from file

In [ ]:
import pickle

with open('dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)


## Step 4: Configure Hyperparameters
Set up GPU/CPU detection (code automatically detects and use GPU if available).

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} ({'GPU' if device.type == 'cuda' else 'CPU'})")

Define all training parameters in a single configuration dictionary.

In [ ]:

CONFIG = {
    'name': 'yolo_hardhat',
    'model': 'yolo11m.pt',  # Model size options: n, s, m, l, x
    'data': dataset.location + "/data.yaml",
    'epochs': int(os.getenv("MODEL_EPOCHS")),
    'batch': int(os.getenv("MODEL_BATCH")), 
    'imgsz': 640,
    'patience': 15,
    'device': device,
    
    # Optimizer settings
    'optimizer': 'SGD',
    'lr0': 0.001,
    'lrf': 0.005,
    'momentum': 0.9,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_bias_lr': 0.01,
    'warmup_momentum': 0.8,
    'amp': False,
    
    # Data augmentation settings
    'augment': True,
    'hsv_h': 0.015,  # HSV-Hue augmentation
    'hsv_s': 0.7,    # HSV-Saturation augmentation
    'hsv_v': 0.4,    # HSV-Value augmentation
    'degrees': 10,    # Image rotation (+/- deg)
    'translate': 0.1, # Image translation
    'scale': 0.3,    # Image scale
    'shear': 0.0,    # Image shear
    'perspective': 0.0,  # Image perspective
    'flipud': 0.1,   # Flip up-down
    'fliplr': 0.1,   # Flip left-right
    'mosaic': 1.0,   # Mosaic augmentation
    'mixup': 0.0,    # Mixup augmentation
}

# Configure PyTorch for GPU memory allocation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

## Step 5: Load Model
Initialize the YOLO model.

In [ ]:
model = YOLO(CONFIG['model'])

## Step 6: Start Training

Begin the training process. By default, the `train` method handles both "training" and "validation" sets. 

In [ ]:
results_train = model.train(
    name=CONFIG['name'],
    data=CONFIG['data'],
    epochs=CONFIG['epochs'],
    batch=CONFIG['batch'],
    imgsz=CONFIG['imgsz'],
    patience=CONFIG['patience'],
    device=CONFIG['device'],
    verbose=True,
    
    # Optimizer parameters
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    momentum=CONFIG['momentum'],
    weight_decay=CONFIG['weight_decay'],
    warmup_epochs=CONFIG['warmup_epochs'],
    warmup_bias_lr=CONFIG['warmup_bias_lr'],
    warmup_momentum=CONFIG['warmup_momentum'],
    amp=CONFIG['amp'],
    
    # Augmentation parameters
    augment=CONFIG['augment'],
    hsv_h=CONFIG['hsv_h'],
    hsv_s=CONFIG['hsv_s'],
    hsv_v=CONFIG['hsv_v'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    shear=CONFIG['shear'],
    perspective=CONFIG['perspective'],
    flipud=CONFIG['flipud'],
    fliplr=CONFIG['fliplr'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
)

Serialize variable into a file

In [ ]:
results_train_serializable = {
    "maps": results_train.maps,
    "names": results_train.names,
    "save_dir": results_train.save_dir,
    "results_dict": results_train.results_dict,
}

results_train_save_path = "model_train_results.pth"

torch.save(results_train_serializable, results_train_save_path)

## Step 7: Evaluate Model

 Evaluate the model on the test set.

In [ ]:
results_test = model.val(data=CONFIG['data'], split='test', device=CONFIG['device'], imgsz=CONFIG['imgsz'])

#print("Test Results:", results_test)

Serialize variable into a file

In [ ]:
results_test_serializable = {
    "maps": results_test.maps,
    "names": results_test.names,
    "save_dir": results_test.save_dir,
    "results_dict": results_test.results_dict,
}

results_test_save_path = "model_test_results.pth"

torch.save(results_test_serializable, results_test_save_path)

## Step 8: (Optional) Model Export

Export the trained YOLO model to ONNX format for deployment.

In [ ]:
model.export(format='onnx', imgsz=CONFIG['imgsz'])

Export the trained YOLO model to TorchScript

In [ ]:
#model.export(format="torchscript")